In [17]:
import json
dataset_name = 'qnli'
with open(f'dy_log/{dataset_name}/bert-base-cased/three_regions_data_indices.json' ,'r') as f:
    d = json.loads(f.read())

In [2]:
print(len(d['hard']), len(d['easy']))
assert len(set(d['hard']).intersection(set(d['easy']))) == 0

52371 52371


In [3]:
import datasets
from datasets import load_dataset
data = load_dataset('glue','qnli')

Reusing dataset glue (/home/v-biyangguo/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
easy_data = data['train'].select(d['easy'])
hard_data = data['train'].select(d['hard'])

In [5]:
easy_data = easy_data.remove_columns(['label'])
easy_data = easy_data.add_column('label', [0]*len(easy_data))

hard_data = hard_data.remove_columns(['label'])
hard_data = hard_data.add_column('label', [1]*len(easy_data))

print(set(easy_data['label']), set(hard_data['label']))

Flattening the indices:   0%|          | 0/53 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/53 [00:00<?, ?ba/s]

{0} {1}


In [9]:
easy_data

Dataset({
    features: ['question', 'sentence', 'idx', 'label'],
    num_rows: 52371
})

In [14]:
import random
easy_indices = d['easy']
random.shuffle(easy_indices)

hard_indices = d['hard']
random.shuffle(hard_indices)

easy_data_train = easy_data.filter(lambda x: x['idx'] in easy_indices[:int(0.7*len(easy_indices))])
easy_data_valid = easy_data.filter(lambda x: x['idx'] in easy_indices[int(0.7*len(easy_indices)):])

hard_data_train = hard_data.filter(lambda x: x['idx'] in hard_indices[:int(0.7*len(hard_indices))])
hard_data_valid = hard_data.filter(lambda x: x['idx'] in hard_indices[int(0.7*len(hard_indices)):])

assert len(set(easy_data_train['idx']).intersection(set(easy_data_valid['idx']))) == 0
assert len(set(hard_data_train['idx']).intersection(set(hard_data_valid['idx']))) == 0


In [15]:
easy_data_train, easy_data_valid

(Dataset({
     features: ['question', 'sentence', 'idx', 'label'],
     num_rows: 36659
 }),
 Dataset({
     features: ['question', 'sentence', 'idx', 'label'],
     num_rows: 15712
 }))

In [16]:
new_train_set = datasets.concatenate_datasets([easy_data_train, hard_data_train]).shuffle(seed=1)
new_valid_set = datasets.concatenate_datasets([easy_data_valid, hard_data_valid]).shuffle(seed=1)

In [24]:
new_train_set, new_valid_set

(Dataset({
     features: ['sentence', 'idx', 'label'],
     num_rows: 47142
 }),
 Dataset({
     features: ['sentence', 'idx', 'label'],
     num_rows: 20206
 }))

In [18]:
new_train_set.to_csv(f'datasets/{dataset_name}-easy-hard_train.csv')
new_valid_set.to_csv(f'datasets/{dataset_name}-easy-hard_valid.csv')

Creating CSV from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

7585188

In [19]:
# 读取
from datasets import load_dataset
data_files = {}
data_files["train"] = f'datasets/{dataset_name}-easy-hard_train.csv'
data_files["validation"] = f'datasets/{dataset_name}-easy-hard_valid.csv'
extension = 'csv'
raw_datasets = load_dataset(extension, data_files=data_files)
raw_datasets

Using custom data configuration default-36cdcd5cd5588345


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/v-biyangguo/.cache/huggingface/datasets/csv/default-36cdcd5cd5588345/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'question', 'sentence', 'idx', 'label'],
        num_rows: 73318
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'question', 'sentence', 'idx', 'label'],
        num_rows: 31424
    })
})